# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Import the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the experiment slice

The following creates three nodes with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "P4DPDK_SYN"

In [2]:
slice = fablib.new_slice(name="P4DPDK_HH4")

### Step 3.2: Define the sites
The code below requests three random sites from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 1 SmartNIC</li>
    <li> 8 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
# sites= fablib.get_random_sites(count=3, filter_function=lambda x: x['nic_connectx_6_available'] > 1 and x['cores_available'] > 8 and x['ram_available'] > 8 and x['disk_available'] > 20)
sites = ['GATECH','SEAT','ATLA']
print (f'The selected sites are {sites[0]}, {sites[1]} and {sites[2]}')

The selected sites are GATECH, SEAT and ATLA


In [4]:
server1 = slice.add_node(name="server1", 
                      site=sites[0], 
                      cores=32, 
                      ram=64, 
                      disk=60, 
                      image='default_ubuntu_20')
server2 = slice.add_node(name="server2", 
                      site=sites[0], 
                      cores=32, 
                      ram=64, 
                      disk=60, 
                      image='default_ubuntu_20')

switch = slice.add_node(name="switch", 
                      site=sites[2], 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

server3 = slice.add_node(name="server3", 
                      site=sites[2], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server4 = slice.add_node(name="server4", 
                      site=sites[2], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server5 = slice.add_node(name="server5", 
                      site=sites[2], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server6 = slice.add_node(name="server6", 
                      site=sites[2], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

In [5]:
server1_iface = server1.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]

server2_1_iface = server2.add_component(model='NIC_ConnectX_6', name='nic2').get_interfaces()[0]
server2_s_iface = server2.get_component(name='nic2').get_interfaces()[1]

switch_2_iface = switch.add_component(model='NIC_ConnectX_6', name='nicS2').get_interfaces()[0]
# server2_s_iface = server2.get_component(name='nic2').get_interfaces()[1]_s_iface = server2.get_component(name='nic2').get_interfaces()[1]
switch_3_iface = switch.add_component(model='NIC_Basic', name='nicS3').get_interfaces()[0]
switch_4_iface = switch.add_component(model='NIC_Basic', name='nicS4').get_interfaces()[0]
switch_5_iface = switch.add_component(model='NIC_Basic', name='nicS5').get_interfaces()[0]
switch_6_iface = switch.add_component(model='NIC_Basic', name='nicS6').get_interfaces()[0]

server3_iface = server3.add_component(model='NIC_Basic', name='nic3').get_interfaces()[0]
server4_iface = server4.add_component(model='NIC_Basic', name='nic4').get_interfaces()[0]
server5_iface = server5.add_component(model='NIC_Basic', name='nic5').get_interfaces()[0]
server6_iface = server6.add_component(model='NIC_Basic', name='nic6').get_interfaces()[0]

### Step 3.5: Connecting server1 and server2
Create a network between server1 and server2 connecting them together and a network between server2 and server3 connecting them together.

<img src="./labs_files/SYN/figs/05_connecting_nodes.png" width="550"><br>

In [6]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, server2_1_iface])
net2 = slice.add_l2network(name='net2', interfaces=[server2_s_iface, switch_2_iface])
net3 = slice.add_l2network(name='net3', interfaces=[server3_iface, switch_3_iface])
net4 = slice.add_l2network(name='net4', interfaces=[server4_iface, switch_4_iface])
net5 = slice.add_l2network(name='net5', interfaces=[server5_iface, switch_5_iface])
net6 = slice.add_l2network(name='net6', interfaces=[server6_iface, switch_6_iface])

### Step 3.6: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [28]:
slice.submit();

KeyboardInterrupt: 

# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the Mellanox drivers, DPDK library, the P4 compiler (p4c), and all needed dependencies.

## Step 4.1 Appending list of servers
All servers are appended to a list to execute commands in parallel

In [8]:
servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))
servers.append(slice.get_node(name="switch"))     
servers.append(slice.get_node(name="server3"))
servers.append(slice.get_node(name="server4"))     
servers.append(slice.get_node(name="server5"))     
servers.append(slice.get_node(name="server6"))     

server1 = servers[0]
server2 = servers[1]
switch = servers[2]
server3 = servers[3]
server4 = servers[4]
server5 = servers[5]
server6 = servers[6]

## Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [9]:
from ipaddress import ip_address, IPv6Address

threads = []

for server in servers:
    if type(ip_address(server.get_management_ip())) is IPv6Address:
        server.upload_file('scripts/nat64.sh', 'nat64.sh')
        threads.append(server.execute_thread(f'chmod +x nat64.sh && ./nat64.sh'))

for thread in threads:
    thread.result()

## Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [10]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo apt-get update;
        sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools hping3;
        sudo pip3 install meson ninja
    '''))

for thread in threads:
    thread.result()

## Step 4.4 Installing Mellanox drivers
Since ConnectX-6 NICs are used in this lab, it is essential to install the supporting drivers. The code below downloads and installs Mellanox drivers on all servers while enabling DPDK

In [11]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64; 
        echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update --enable-sriov --hypervisor
    '''))
    
for thread in threads:
    thread.result()

## Step 4.5 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers

In [12]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        git clone http://dpdk.org/git/dpdk; 
        cd dpdk;
        sudo meson build;
        cd build;
        sudo ninja;
        sudo ninja install; 
        sudo ldconfig
    '''))

for thread in threads:
    thread.result()

In [13]:
# stdout, stderr = server2.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
# server2.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
# server2.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
# stdout, stderr = server2.execute(f'cd dpdk && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

## Step 4.6 Installing Pktgen
In this lab, we will send from server1 packets at a high rate to server2 using a DPDK-based packet generation tool called pktgen [<a href="#References">5</a>]. The code below downloads and installs Pktgen-DPDK on all servers

In [13]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo git clone https://github.com/pktgen/Pktgen-DPDK; 
        sudo sed -i \"s/deps += \\[dependency('numa', required: true)\\]/deps += \\[dependency('numa', required: false)\\]/\" /home/ubuntu/Pktgen-DPDK/app/meson.build;
        sudo apt-get install -y cmake libpcap-dev libbsd-dev;
        cd Pktgen-DPDK &&  sudo meson build && sudo ninja -C build && cd build/ && sudo meson install
    '''))

for thread in threads:
    thread.result()

## Step 4.7 Build pipeline library
The code below builds the DPDK pipeline library in server 2 (on which the pipeline will be running) to put all its functions into effect 

In [14]:
stdout, stderr = server2.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

## Step 4.8 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline. In this lab, p4c is built from a version where the architecture has been modified.

In [15]:
stdout, stderr = server2.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server2.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server2.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

Switch

In [17]:
switch = slice.get_node(name="switch")     
switch.upload_file('main/scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.9 Reboot
After installing the ConnectX Mellanox divers, it is essential to reboot the servers to complete the installation process or to ensure that updates are applied correctly

In [18]:
for server in servers:
    server.os_reboot()

# Step 5: Configuring Network
In this step, we will assign IPv4 addresses to the interfaces of the servers and hardcode the MAC addresses. We will also configure forwarding and routing.

In [13]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()+'np0'
print(f'server1_iface: {server1_iface_name}')

node2_iface1 = server2.get_interface(network_name='net1') 
server2_iface1_name = node2_iface1.get_device_name()+'np0'
print(f'server2_iface1: {server2_iface1_name}')
node2_iface2 = server2.get_interface(network_name='net2') 
server2_iface2_name = node2_iface2.get_device_name()+'np1'
print(f'server2_iface2: {server2_iface2_name}')

switch_iface = switch.get_interface(network_name='net2') 
switch_iface_name = switch_iface.get_device_name()+'np0'
print(f'switch_iface: {switch_iface_name}')
switch_iface3 = switch.get_interface(network_name='net3') 
switch_iface3_name = switch_iface3.get_device_name()+'np0'
print(f'switch_iface3: {switch_iface3_name}')
switch_iface4 = switch.get_interface(network_name='net4') 
switch_iface4_name = switch_iface4.get_device_name()+'np0'
print(f'switch_iface4: {switch_iface4_name}')
switch_iface5 = switch.get_interface(network_name='net5') 
switch_iface5_name = switch_iface5.get_device_name()+'np0'
print(f'switch_iface5: {switch_iface5_name}')
switch_iface6 = switch.get_interface(network_name='net6') 
switch_iface6_name = switch_iface6.get_device_name()+'np0'
print(f'switch_iface6: {switch_iface6_name}')

node3_iface = server3.get_interface(network_name='net3') 
server3_iface_name = node3_iface.get_device_name()+'np0'
print(f'server3_iface: {server3_iface_name}')

node4_iface = server4.get_interface(network_name='net4') 
server4_iface_name = node4_iface.get_device_name()+'np0'
print(f'server4_iface: {server4_iface_name}')

node5_iface = server5.get_interface(network_name='net5') 
server5_iface_name = node5_iface.get_device_name()+'np0'
print(f'server5_iface: {server5_iface_name}')

node6_iface = server6.get_interface(network_name='net6') 
server6_iface_name = node6_iface.get_device_name()+'np0'
print(f'server6_iface: {server6_iface_name}')

server1_iface: enp7s0np0
server2_iface1: enp7s0np0
server2_iface2: enp8s0np1
switch_iface: enp9s0np0
switch_iface3: enp7s0np0
switch_iface4: enp11s0np0
switch_iface5: enp8s0np0
switch_iface6: enp12s0np0
server3_iface: enp7s0np0
server4_iface: enp7s0np0
server5_iface: enp7s0np0
server6_iface: enp7s0np0


In [14]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface1_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface2_name} up', quiet=True)

stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface3_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface4_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface5_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface6_name} up', quiet=True)

stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)
stdout, stderr = server4.execute(f'sudo ip link set dev {server4_iface_name} up', quiet=True)
stdout, stderr = server5.execute(f'sudo ip link set dev {server5_iface_name} up', quiet=True)
stdout, stderr = server6.execute(f'sudo ip link set dev {server6_iface_name} up', quiet=True)

In [15]:
server1_iface_MAC = '00:00:00:00:00:01' 
server2_iface1_MAC = '00:00:00:00:00:21' 
server2_iface2_MAC = '00:00:00:00:00:22' 

switch_iface_MAC = '00:00:00:00:00:30' 
switch_iface3_MAC = '00:00:00:00:00:33'
switch_iface4_MAC = '00:00:00:00:00:34'
switch_iface5_MAC = '00:00:00:00:00:35'
switch_iface6_MAC = '00:00:00:00:00:36'

server3_iface_MAC = '00:00:00:00:00:03'
server4_iface_MAC = '00:00:00:00:00:04'
server5_iface_MAC = '00:00:00:00:00:05'
server6_iface_MAC = '00:00:00:00:00:06'

In [17]:
stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} hw ether {server2_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} hw ether {server2_iface2_MAC}')

stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface_name} hw ether {switch_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface3_name} hw ether {switch_iface3_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface4_name} hw ether {switch_iface4_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface5_name} hw ether {switch_iface5_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface6_name} hw ether {switch_iface6_MAC}')

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')
stdout, stderr = server4.execute(f'sudo ifconfig {server4_iface_name} hw ether {server4_iface_MAC}')
stdout, stderr = server5.execute(f'sudo ifconfig {server5_iface_name} hw ether {server5_iface_MAC}')
stdout, stderr = server6.execute(f'sudo ifconfig {server6_iface_name} hw ether {server6_iface_MAC}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server4: Name or service not known
  sudo: unable to resolve host server5: Name or service not known
  sudo: unable to resolve host server6: Name or service not known
 

In [22]:
server1_iface_IP = '192.168.10.01/24' 
server2_iface1_IP = '192.168.10.21/24' 
server2_iface2_IP = '192.168.20.22/24' 

switch_iface_IP = '192.168.20.30/24' 
switch_iface3_IP = '192.168.30.33/24'
switch_iface4_IP = '192.168.30.34/24'
switch_iface5_IP = '192.168.30.35/24'
switch_iface6_IP = '192.168.30.36/24'

server3_iface_IP = '192.168.30.03/24'
server4_iface_IP = '192.168.30.04/24'
server5_iface_IP = '192.168.30.05/24'
server6_iface_IP = '192.168.30.06/24'

net1_subnet = "192.168.10.0/24"
net2_subnet = "192.168.20.0/24"
net3_subnet = "192.168.30.0/24"

In [23]:
stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_iface_IP}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} {server2_iface1_IP}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} {server2_iface2_IP}')

stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface_name} {switch_iface_IP}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface3_name} {switch_iface3_IP}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface4_name} {switch_iface4_IP}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface5_name} {switch_iface5_IP}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface6_name} {switch_iface6_IP}')

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_iface_IP}')
stdout, stderr = server4.execute(f'sudo ifconfig {server4_iface_name} {server4_iface_IP}')
stdout, stderr = server5.execute(f'sudo ifconfig {server5_iface_name} {server5_iface_IP}')
stdout, stderr = server6.execute(f'sudo ifconfig {server6_iface_name} {server6_iface_IP}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server4: Name or service not known
  sudo: unable to resolve host server5: Name or service not known
  sudo: unable to resolve host server6: Name or service not known
 

In [29]:
stdout, stderr = server1.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = server2.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = server3.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = server4.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = server5.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = server5.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
stdout, stderr = switch.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)

In [26]:
ip1 = server1_iface_IP.split('/')[0]
ip2_1 = server2_iface1_IP.split('/')[0]
ip2_2 = server2_iface2_IP.split('/')[0]
ips = switch_iface_IP.split('/')[0]
ips_3 = switch_iface3_IP.split('/')[0]
ips_4 = switch_iface4_IP.split('/')[0]
ips_5 = switch_iface5_IP.split('/')[0]
ips_6 = switch_iface6_IP.split('/')[0]
ip3 = server3_iface_IP.split('/')[0]
ip4 = server4_iface_IP.split('/')[0]
ip5 = server5_iface_IP.split('/')[0]
ip6 = server6_iface_IP.split('/')[0]
subnet1 = "192.168.10.0/24"
subnet2 = "192.168.20.0/24"
subnet3 = "192.168.30.0/24"

stdout, stderr = server1.execute(f'sudo arp -s {ip2_1} {server2_iface1_MAC} -i {server1_iface_name}')
stdout, stderr = server1.execute(f'sudo ip route add {subnet2} via {ip2_1}')
stdout, stderr = server1.execute(f'sudo ip route add {subnet3} via {ip2_1}')

stdout, stderr = server2.execute(f'sudo arp -s {ip1} {server1_iface_MAC} -i {server2_iface1_name}')
stdout, stderr = server2.execute(f'sudo arp -s {ips} {switch_iface_MAC} -i {server2_iface2_name}')
stdout, stderr = server2.execute(f'sudo ip route add {subnet3} via {ips}')

ststdout, stderr = switch.execute(f'sudo arp -s {ip2_2} {server2_iface2_MAC} -i {switch_iface_name}')
stdout, stderr = switch.execute(f'sudo arp -s {ip3} {server3_iface_MAC} -i {switch_iface3_name}')
stdout, stderr = switch.execute(f'sudo arp -s {ip4} {server4_iface_MAC} -i {switch_iface4_name}')
stdout, stderr = switch.execute(f'sudo arp -s {ip5} {server5_iface_MAC} -i {switch_iface5_name}')
stdout, stderr = switch.execute(f'sudo arp -s {ip6} {server6_iface_MAC} -i {switch_iface6_name}')
stdout, stderr = switch.execute(f'sudo ip route add {subnet1} via {ip2_2}')
stdout, stderr = switch.execute(f'sudo ip route add {ip3} dev {switch_iface3_name} src {ips_3}')
stdout, stderr = switch.execute(f'sudo ip route add {ip4} dev {switch_iface4_name} src {ips_4}')
stdout, stderr = switch.execute(f'sudo ip route add {ip5} dev {switch_iface5_name} src {ips_5}')
stdout, stderr = switch.execute(f'sudo ip route add {ip6} dev {switch_iface6_name} src {ips_6}')

stdout, stderr = server3.execute(f'sudo arp -s {ips_3} {switch_iface3_MAC} -i {server3_iface_name}')
stdout, stderr = server3.execute(f'sudo ip route add {subnet1} via {ips_3}')
stdout, stderr = server3.execute(f'sudo ip route add {subnet2} via {ips_3}')

stdout, stderr = server4.execute(f'sudo arp -s {ips_4} {switch_iface4_MAC} -i {server4_iface_name}')
stdout, stderr = server4.execute(f'sudo ip route add {subnet1} via {ips_4}')
stdout, stderr = server4.execute(f'sudo ip route add {subnet2} via {ips_4}')

stdout, stderr = server5.execute(f'sudo arp -s {ips_5} {switch_iface5_MAC} -i {server5_iface_name}')
stdout, stderr = server5.execute(f'sudo ip route add {subnet1} via {ips_5}')
stdout, stderr = server5.execute(f'sudo ip route add {subnet2} via {ips_5}')

stdout, stderr = server6.execute(f'sudo arp -s {ips_6} {switch_iface6_MAC} -i {server6_iface_name}')
stdout, stderr = server6.execute(f'sudo ip route add {subnet1} via {ips_6}')
stdout, stderr = server6.execute(f'sudo ip route add {subnet2} via {ips_6}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
RTNETLINK answers: File exists
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host switch: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
RTNETLINK answers: File exists
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
  sudo: unable to re

## Step 5.8: Mellanox devices

In this step, we will inspect and start all Mellanox devices.

In [27]:
stdout, stderr = server1.execute(f'sudo ibdev2netdev')
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)
stdout, stderr = server1.execute(f'sudo mst start', quiet=True)
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)

stdout, stderr = server2.execute(f'sudo ibdev2netdev')
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)
stdout, stderr = server2.execute(f'sudo mst start', quiet=True)
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)

stdout, stderr = switch.execute(f'sudo ibdev2netdev')
stdout, stderr = switch.execute(f'sudo mst status', quiet=True)
stdout, stderr = switch.execute(f'sudo mst start', quiet=True)
stdout, stderr = switch.execute(f'sudo mst status', quiet=True)

stdout, stderr = server3.execute(f'sudo ibdev2netdev')
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)
stdout, stderr = server3.execute(f'sudo mst start', quiet=True)
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)

stdout, stderr = server4.execute(f'sudo ibdev2netdev')
stdout, stderr = server4.execute(f'sudo mst status', quiet=True)
stdout, stderr = server4.execute(f'sudo mst start', quiet=True)
stdout, stderr = server4.execute(f'sudo mst status', quiet=True)

stdout, stderr = server5.execute(f'sudo ibdev2netdev')
stdout, stderr = server5.execute(f'sudo mst status', quiet=True)
stdout, stderr = server5.execute(f'sudo mst start', quiet=True)
stdout, stderr = server5.execute(f'sudo mst status', quiet=True)

stdout, stderr = server6.execute(f'sudo ibdev2netdev')
stdout, stderr = server6.execute(f'sudo mst status', quiet=True)
stdout, stderr = server6.execute(f'sudo mst start', quiet=True)
stdout, stderr = server6.execute(f'sudo mst status', quiet=True)

mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server1: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Up)
 sudo: unable to resolve host server2: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np0 (Up)
 sudo: unable to resolve host switch: Name or service not known
 mlx5_2 port 1 ==> enp9s0np0 (Up)
mlx5_3 port 1 ==> enp10s0np1 (Down)
mlx5_4 port 1 ==> enp11s0np0 (Up)
mlx5_5 port 1 ==> enp12s0np0 (Up)
mlx5_0 port 1 ==> enp7s0np0 (Up)
 sudo: unable to resolve host server3: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
 sudo: unable to resolve host server4: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
 sudo: unable to resolve host server5: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
 sudo: unable to resolve host server6: Name or service not known
 

In the output above, you can see that there are two interfaces that are turned down. This is because each server has a Connect-X6 NIC attached, which is a dual-port NIC, and only one port in the NICs of server1 and server3 is used in this topology. 

# Step 6: Implementing the P4 code

## Step 6.1: Modifying the control file

Click on [control.p4](./labs_files/SYN/control.p4) to open the CLI file in the editor.

<img src="./labs_files/SYN/figs/6_01_00.png" width="650"><br>
We can see that the control.p4 declares a control block named MainControl. Note that we will modify the body of the control block.

<hr>

In the control.p4 file, define the variable ```THRESH```. ```THRESH``` represents the threshold of received SYN packets per second after which the pipeline will start dropping packets. The threshold is set to 1000000 packets.

    #define THRESH 1000000

<img src="./labs_files/SYN/figs/6_01_01.png" width="650"><br>

<hr>

In the MainControl block, define the register ```drop_percent_reg``` to maintain the percentage of packets to be dropped after the number of received SYN packets increases above the threshold.

    Register<bit<7>, bit<1>>(1) drop_percent_reg;

<img src="./labs_files/SYN/figs/6_01_02.png" width="650"><br>

The ```Register``` extern creates a register by taking the cell width in bits, 7 bits in this example. The second argument considered by this extern is the number of bits needed to represent the indices of the register, 1 bit in this example. One bit is enough to represent all indices in this register because the length of the register is specified as 1. Finally, the register is associated with a label ```drop_percent_reg```.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
Note that the p4c-dpdk compiler adds (_0) to the labels of the registers defined in the P4 code. Therefore, drop_percent_reg is compiled as drop_percent_reg_0 in the pipeline. </div>

The code above defines a register named drop_percent_reg. The register contains a single cell. The cell stores the percentage of packets to be dropped. This register can be configured from the control plane at runtime to specify the dropping percentage. Because the maximum possible dropping percentage is 100, and 7 is the minimum number of bits needed to represent 100 (since 2^7=128), the size of the register cell is set to 7 bits. 

<hr>

Define the register syn_counts_reg to maintain the count of the received SYN packets.

    Register<bit<32>, bit<1>>(1) syn_counts_reg;

<img src="./labs_files/SYN/figs/6_01_03.png" width="650"><br>

The code above defines a register named ```syn_counts_reg```. The register contains a single cell. The cell stores the number of received SYN packets. Later, we will be resetting the value of this register to zero; thus, this cell will contain the number of received SYN packets per second. 

<hr>

Define the register percent_iterator_reg to maintain the packet count iterator.

    Register<bit<7>, bit<1>>(1) percent_iterator_reg;

<img src="./labs_files/SYN/figs/6_01_04.png" width="650"><br>

The code above defines a register named ```percent_iterator_reg```. The register contains a single cell. This cell is used to track how many packets to drop and to allow out of 100.

<hr>

Add the following code to the apply block to retrieve the dropping percentage from the register.

    if(hdr.tcp.isValid()){
        meta.drop_percent = drop_percent_reg.read(0);
    }

<img src="./labs_files/SYN/figs/6_01_05.png" width="650"><br>

The code above defines a register named ```percent_iterator_reg```. The register contains a single cell. This cell is used to track how many packets to drop and to allow out of 100.

In the code above, ```if(hdr.tcp.isValid())``` checks if the packet is a TCP packet. For TCP packets, the dropping percentage is retrieved from ```drop_percentage_reg``` and stored in the ```drop_percent``` variable. Note that the dropping percentage will be specified from the control plane.

<hr>

Add the following code to check if the incoming packet is a SYN packet.

    if(hdr.tcp.flags == 2) {
    
    }

<img src="./labs_files/SYN/figs/6_01_06.png" width="650"><br>

<hr>

Add the following code to increment the count of SYN packets. 
    
    meta.syn_counts = 0;

    meta.syn_counts = syn_counts_reg.read(0);
    meta.syn_counts = meta.syn_counts +1;
    syn_counts_reg.write(0, meta.syn_counts);

<img src="./labs_files/SYN/figs/6_01_07.png" width="650"><br>

```.read``` is a method used to read the value in a register at a specific index. ```.write``` is a method used to write a value to a register at a specific index. In the code above, the count of SYN packets is retrieved from the ```syn_counts_reg``` at index 0 and stored in the ```syn_counts``` variable. The variable is incremented by one to account for the current packet. After that, the updated ```syn_counts``` variable is stored in the ```syn_count_reg``` register at index 0.

<hr>

Add the following code to check if the number of SYN packets exceeded THRESH. 

    if(meta.syn_counts > THRESH){

    }

<img src="./labs_files/SYN/figs/6_01_08.png" width="650"><br>

<hr>

Add the following code to retrieve the iterator from the ```percent_iterator_reg```.

    meta.percent_iterator = percent_iterator_reg.read(0);

<img src="./labs_files/SYN/figs/6_01_09.png" width="650"><br>

In the code above, the iterator is retrieved from ```percent_iterator_reg``` and stored inside the ```percent_iterator``` variable.

<hr>

Add the following code to check if the iterator is less than the dropping percentage. 

    if(meta.percent_iterator < meta.drop_percent){

    }

<img src="./labs_files/SYN/figs/6_01_10.png" width="650"><br>

<hr>

Add the following code to drop the packet and increment the iterator if the ```percent_iterator``` is less than the ```drop_percent```.

    meta.percent_iterator = meta.percent_iterator + 1;
    percent_iterator_reg.write(0, meta.percent_iterator);
    drop();

<img src="./labs_files/SYN/figs/6_01_11.png" width="650"><br>

In the code above, the ```percent_iterator``` variable is incremented by one and stored in the ```percent_iterator_reg``` register. After that, the packet is dropped.

<hr>

Add the following code to increment the count of dropped packets by one without dropping the packet if the number of dropped packet is less than 100.

    else if (meta.percent_iterator < 100) {
       meta.percent_iterator = meta.percent_iterator + 1;
       percent_iterator_reg.write(0, meta.percent_iterator);
    }

<img src="./labs_files/SYN/figs/6_01_12.png" width="650"><br>

From each 100 packets, we are dropping the first ```drop_percent``` packets (e.g., the first 50 packets if the ```drop_percen```t is 50%). The remaining packets (i.e., 100 – ```drop_percent```) are forwarded.

<hr>

Add the following code to reset ```percent_iterator_reg``` register when ```percent_iterator``` reaches 100.

    else if (meta.percent_iterator == 100) {
        meta.percent_iterator = 0;
        percent_iterator_reg.write(0, meta.percent_iterator);
    }

<img src="./labs_files/SYN/figs/6_01_13.png" width="650"><br>

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 6.2: Modifying the headers file

Click on [headers.p4](./labs_files/SYN/headers.p4) to open the I/O file in the editor.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
In the PNA architecture of P4, variables needed in the body of the control block will have to be declared as metadata. </div>

Associate each declared register with a variable in the metadata structure by adding the following code.

    bit<7> drop_percent;
    bit<32> syn_counts;
    bit<7> percent_iterator;

<img src="./labs_files/SYN/figs/6_02_00.png" width="650"><br>

<hr>

Save the changes by pressing ```Ctrl+s```.

# Step 7: Compiling the P4 program

In this lab, we will not modify the P4 code in which we implement a simple packet reflector. Instead, we will just compile it using the p4c-dpdk compiler. Note that in this P4 code the Portable NIC Architecture (PNA) is used.

To upload and compile the P4 program, issue the following command.

In [1]:
# # server2.upload_file('labs_files/SYN/headers.p4','headers.p4')
# server2.upload_file('scripts/run/SYN/headers.p4','headers.p4')
# server2.upload_file('labs_files/SYN/parser.p4','parser.p4')
# server2.upload_file('labs_files/SYN/precontrol.p4','precontrol.p4')
# # server2.upload_file('labs_files/SYN/control.p4','control.p4')
# server2.upload_file('scripts/run/SYN/control.p4','control.p4')
# server2.upload_file('labs_files/SYN/deparser.p4','deparser.p4')
# server2.upload_file('labs_files/SYN/main.p4','main.p4')
# stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o syn.spec')
# stdout, stderr = server2.execute(f'ls')

In [26]:
server2.upload_file('experiments/reflector/reflector.p4','reflector.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna reflector.p4 -o reflector.spec')
server2.upload_file('experiments/reflector/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/reflector/reflector.cli','reflector.cli')

 sudo: unable to resolve host server2: Name or service not known
 

<SFTPAttributes: [ size=556 uid=1000 gid=1000 mode=0o100664 atime=1729191130 mtime=1729191131 ]>

The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab9.spec``` file which is a specification file needed to run the pipeline.

# Step 8: Running the P4-DPDK pipeline
Now that all the required scripts are prepared, we can run the pipeline.

## Step 8.1: Uploading files
The following code uploads the CLI and I/O scripts to server1 and server2.

In [26]:
server2.upload_file('labs_files/SYN/syn.cli','syn.cli')
server2.upload_file('labs_files/SYN/ethdev0.io','ethdev0.io')
server2.upload_file('labs_files/SYN/ethdev1.io','ethdev1.io')
server2.upload_file('labs_files/SYN/ethdev2.io','ethdev2.io')
server2.upload_file('labs_files/SYN/ethdev3.io','ethdev3.io')
server2.upload_file('labs_files/SYN/rules.txt','rules.txt')
server2.upload_file('labs_files/SYN/reset_SYN_packets_per_second.py','reset_SYN_packets_per_second.py')

<SFTPAttributes: [ size=1183 uid=1000 gid=1000 mode=0o100664 atime=1729001524 mtime=1729001524 ]>

## Step 8.2: Reserving hugepages
Configure the number of hugepages in the system by typing the following command. 

In [27]:
threads = []

for server in servers:
    threads.append(server.execute_thread(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"'))

for thread in threads:
    thread.result()

Hugepage reservation is done by setting the number of hugepages required to the ```nr_hugepages``` file in the kernel corresponding to a specific page size (in Kilobytes).

The ```echo``` command is used to print a value which in this case is ```1024``` representing the number of hugepages. The ```>``` symbol is a redirection operator that redirects the output of the previous command (echo 1024) to the file specified in the following path: ```/sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages```

## Step 8.3: Opening a terminal

Launch a new terminal by opening a new tab and then select "terminal".

Copy the output of the command below and paste it in the terminal to enter to server2.

In [2]:
server2.get_ssh_command()

NameError: name 'server2' is not defined

## Step 8.4: Running the pipeline

Run the following commands in the terminal:
    
    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/syn.cli
    
<img src="./labs_files/SYN/figs/pipeline.png" width="900"><br>

In the figure above, the command is used to run the DPDK pipeline application considering the following arguments:

•	```examples/pipeline/build/pipeline```: the path to the executable DPDK pipeline application.<br>
•	```-c```: this parameter is used to specify the hexadecimal bitmask of the cores to run on. In this case, (0x1F) indicated that 4 cores are reserved for the pipelines and one extra core is needed for other processes.<br>
•	```-s```: this parameter is used to specify the path to the CLI script file to be run at application startup ```/home/ubuntu/syn.cli```.<br>

Note that when the DPDK pipeline runs, the CLI script is printed in the terminal. If any problems are encountered while running the pipeline, error messages will be shown within the printed CLI script.


# Setp 9: Testing the application
To test the application, we will send from server1 packets at a high rate to server2 using a DPDK-based packet generation tool called pktgen. 

## Step 9.1: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

Copy the output of the command below and paste it in the terminal to enter to server2.

In [29]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe44:e9de'

## Step 9.2: Reset packet count every one second

Type the command below to reset the count of SYN packets every second.

    sudo python3 reset_SYN_packets_per_second.py
    
<img src="./labs_files/SYN/figs/reset_reg.png" width="800px"><br>

## Step 9.3: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

Copy the output of the command below and paste it in the terminal to enter to server1.

In [30]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe7f:ec1e'

## Step 9.4: Running pktgen

Run the command below to know which interface is being used on server 1 to communicate with server 2 and server 3

In [57]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


The output of this command will be used to refer to the interface ID in the command invoked to run pktgen

Run the following commands in the terminal to run pktgen:

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
It is important that the interface name matches the output of the previous command. This is an example where the interface ID is 07:00.0
</div>

    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"
    
<img src="./labs_files/SYN/figs/pktgen_command.png" width="650"><br>

In the figure above, the command ```pktgen``` is used to run the packet generator considering the following arguments:

•	```-l```: List of cores to run on.<br>
•	```-n```: Number of memory channels.<br>
•	```-a```: The ID of allowed interfaces (the command shows an example where interface 07:00.0 is used).<br>
•	```-P```: Enable promiscuous mode on all ports.<br>
•	```-m```: Matrix for mapping ports to logical cores.<be>

In this step, we are invoking pktgen, using cores 2 cores (0 and 1) as specified in the ```-l``` parameter with 4 memory channels as specified in the ```-n``` parameter. The interface allowed ```-a``` has the ID 07:00.0 in this example. We enabled promiscuous mode ```-P``` and mapped CPU core 1 to handle the rx and tx ports of port 0 as specified in the ```-m``` parameter.

As soon a pktgen starts the main screen is displayed:

<img src="./labs_files/SYN/figs/pktgen_main.png" width="650"><br>

## Step 9.5: Configuring pktgen

In pktgen, we will set the source and destination MAC and IP addresses along with the desired packet size of the generated packets. RSS will distribute packets that belong to the same flow on each of the four different pipelines running on four different cores. Therefore, we will have to generate the packets from a range of flows. To do so, we will generate packets from a range of source and destination IP addresses. 

In pktgen terminal, enter the following command to navigate the page displaying the settings of packets sent from a range of flows:

    page range

<img src="./labs_files/SYN/figs/pktgen_range.png" width="650"><be>

<hr>

Enter the following commands to set the source MAC address:

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01

<img src="./labs_files/SYN/figs/pktgen_srcmac.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the source MAC address of the generated packets sent from port ID 0, we used the ```src mac``` command and since the source MAC addresses are not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```00:00:00:00:00:01``` which is the MAC address of server1. 

<hr>

Enter the following commands to set the destination MAC address:

    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21

<img src="./labs_files/SYN/figs/pktgen_dstmac.png" width="850"><be>

To modify the settings in the rage page we used the ```range``` command. To set the destination MAC address of the generated packets sent from port ID 0, we used the ```dst mac``` command and since the destination MAC addresses are not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```00:00:00:00:00:21``` which is the MAC address of the interface on server2 facing server1. 

<hr>

Enter the following commands to set the source IP address:

    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1

<img src="./labs_files/SYN/figs/pktgen_srcip.png" width="850"><be>

To modify the settings in the rage page we used the ```range``` command. To set the source IP address of the generated packets sent from port ID 0, we used the ```src ip``` command. The starting value ```start``` and minimum value ```min``` are both set as the IP ```1.1.1.1```. The rage will reach a maximum value ```max``` to be ```250.250.250.250``` by incrementing each octet by ```0.0.0.1``` and specified in the ```inc```. 

<hr>

Enter the following commands to set the destination IP address:

    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1

<img src="./labs_files/SYN/figs/pktgen_dstip.png" width="850"><be>

To modify the settings in the rage page we used the ```range``` command. To set the destination IP address of the generated packets sent from port ID 0, we used the ```dst ip``` command and since the destination IP addresses are not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```192.168.20.1``` which is the IP address of the interface on server3.

<hr>

Enter the following commands to set the packet size:

    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500

<img src="./labs_files/SYN/figs/pktgen_size.png" width="850"><be>

To modify the settings in the rage page we used the ```range``` command. To set the size in bytes of the generated packets sent from port ID 0, we used the ```size``` command and since the packet size is not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```1500``` to set the packet size to 1500 bytes. 

<hr>

Enter the following commands to clear all flags in the TCP header and set the SYN flag to 1:

    range 0 tcp flag clr all
    range 0 tcp flag set syn

<img src="./labs_files/SYN/figs/pktgen_tcp.png" width="850"><be>

<hr>

Enter the following command to enable generating packets from a range of flows and going back to the main page:

    enable 0 range
    page main

<img src="./labs_files/SYN/figs/pktgen_enable.png" width="650"><be>

## Step 9.6: Start sending packets

In pktgen terminal execute the following commands to start generating and sending packets from server 1 to server 3:

    start 0

<img src="./labs_files/SYN/figs/pktgen_start.png" width="850"><br>

To start sending packets, we used the ```start``` command followed by the port list. In this case, we are using one port with ID 0. We can observe the rate at which the packets are sent from server 1 to server3. In the grey box, the first two lines display the number of packets sent and received per second, and the last line displays the sending and receiving throughput in Mbyte per second. The result in the screenshot above shows that pktgen is generating around 8 million packets at a rate close to 100 Gbyte per second. These packets are sent to server 3. 

## Step 9.7: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

Copy the output of the command below and paste it in the terminal to enter to server3.

In [31]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:feb3:4195'

## Step 9.8: Running pktgen

Run the command below to know which interface is being used on server 3 to communicate with server 1 and server 2

In [31]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


The output of this command will be used to refer to the interface ID in the command invoked to run pktgen

Run the following commands in the terminal to run pktgen:

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
It is important that the interface name matches the output of the previous command. This is an example where the interface ID is 07:00.0
</div>

    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"
    
<img src="./labs_files/SYN/figs/pktgen_command2.png" width="850"><br>

As soon a pktgen starts the main screen is displayed:

<img src="./labs_files/SYN/figs/pktgen_main2.png" width="850"><br>

The figure above shows that h3 is receiving around 8 million SYN packets per second.

Note that no packets are dropped by the pipeline because the dropping percentage is set
to zero. The drop_percent_reg register by default holds the value 0. We will now configure
the dropping rate from the control plane.

## Step 9.9: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

Copy the output of the command below and paste it in the terminal to enter to server2.

In [32]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe44:e9de'

## Step 9.10: Opening the pipeline CLI

Enter the pipeline CLI by typing the following command.

    telnet 0.0.0.0 8086
    
<img src="./labs_files/SYN/figs/telnet.png" width="700"><br>

The ```telnet``` command is followed by the IP address of the server (0.0.0.0) and the port
number (8086).

## Step 9.11: Configuring the drop rate

Configure the dropping rate to be 50% by typing the command below.

    pipeline PIPELINE0 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 50 index 0
    
<img src="./labs_files/SYN/figs/drop50.png" width="800"><br>

The ```regwr``` command writes a value to the register drop_percent_reg at a specific index.
Therefore, this command takes two inputs ```value``` 50 and the ```index``` 0. 0x32 is the
hexadecimal representation of 50. The following is applied to all pipelines.

Note that the p4c-dpdk compiler adds (_0) to the labels of the registers defined in the P4
code. Therefore, drop_percent_reg is compiled as drop_percent_reg_0 in the pipeline.

## Step 9.12: Inspecting the number of SYN packets

Inspect the number of received SYN packet received by server 3 by observing the pktgen monitor

<img src="./labs_files/SYN/figs/pktgen_drop50.png" width="650"><br>

The figure above shows that h3 is receiving around 4.5 million SYN packets per second.

Because the received number of SYN packets per second is around 8 million, and the threshold
is 1 million, the dropping threshold will be applied to 7 million packets only (8M - 1M). Note that
the pipeline does not apply the dropping mechanism on the first 1 million SYN packets. By
setting the dropping percentage to 50%, we expect to receive 1M + 7M/2 SYN packets,
which is around 4.5 million packets.

## Step 9.13: Configuring the drop rate

In the telnet terminal, configure the dropping rate to be 100% by typing the command below.

    pipeline PIPELINE0 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 100 index 0
    
<img src="./labs_files/SYN/figs/drop100.png" width="700"><br>

By setting the dropping threshold to 100%, the expected number of SYN packets to be
received per second is around 1 million because all the packets above the threshold will be
dropped. 0x64 is the hexadecimal representation of 100.

## Step 9.14: Inspecting the number of SYN packets

Inspect the number of received SYN packet received by server 3 by observing the pktgen monitor

<img src="./labs_files/SYN/figs/pktgen_drop100.png" width="650"><br>

The figure above shows that h3 is receiving around 1 million SYN packets per second.

# Step 10: Delete the slice

This concludes Lab 9. Please delete your slice when you are done with your experiment.

In [3]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_HH3")
# slice.delete()

/opt/conda/lib/python3.11/site-packages/fabric_cm/credmgr/swagger_client/rest.py:45: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  return self.urllib3_response.getheaders()


SliceManagerException: Internal Server Error - (invalid_grant) expired refresh token

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_HH4")
# slice.delete()

In [3]:
servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))
servers.append(slice.get_node(name="switch"))     
servers.append(slice.get_node(name="server3"))
servers.append(slice.get_node(name="server4"))     
servers.append(slice.get_node(name="server5"))     
servers.append(slice.get_node(name="server6"))     

server1 = servers[0]
server2 = servers[1]
switch = servers[2]
server3 = servers[3]
server4 = servers[4]
server5 = servers[5]
server6 = servers[6]

# References

1.	NETSCOUT, “What is a Volumetric Attack?” [Online]. Available: https://www.netscout.com/what-is-ddos/volumetric-attacks
2.	Cloudflare, “SYN Flood Attack.” [Online]. Available: https://www.cloudflare.com/learning/ddos/syn-flood-ddos-attack/
3.	GURU99, “What is TCP Three-Way HandShake?.” [Online]. Available: https://www.guru99.com/tcp-3-way-handshake.html
4.	NETSCOUT, “What is a SYN flood attack and how do you to prevent it?” [Online]. Available: https://www.netscout.com/what-is-ddos/syn-flood-attacks
5.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html